In [1]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC

In [2]:
def preprocess_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    equalized_image = cv2.equalizeHist(blurred_image)
    _, thresholded_image = cv2.threshold(equalized_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return thresholded_image

In [3]:
def preprocess_image(image):
    
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_image)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    clahe_l_channel = clahe.apply(l_channel)
    enhanced_lab_image = cv2.merge((clahe_l_channel, a_channel, b_channel))
    enhanced_bgr_image = cv2.cvtColor(enhanced_lab_image, cv2.COLOR_LAB2BGR)
    blurred_image = cv2.GaussianBlur(enhanced_bgr_image, (5, 5), 0)
    

    return blurred_image

In [4]:
def segment_blood_cells(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    _, segmented_cells = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    kernel = np.ones((3, 3), np.uint8)
    segmented_cells = cv2.morphologyEx(segmented_cells, cv2.MORPH_OPEN, kernel, iterations=2)
    segmented_cells = cv2.morphologyEx(segmented_cells, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    return segmented_cells

In [5]:
def extract_features(image):

    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    features = []
    
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        features.append([area, perimeter])

    features = np.array(features)
    
    return features

In [6]:
def load_data(train_path, test_path, val_path):
    # Initialize lists to store images and labels for training, testing, and validation sets
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    X_val = []
    y_val = []

    # Function to load images and labels from a directory
    def load_images_from_directory(images_path, labels_path):
        images = []
        labels = []
        for filename in os.listdir(images_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):  # Assuming images are in JPG or PNG format
                image_path = os.path.join(images_path, filename)
                label_path = os.path.join(labels_path, filename.split('.')[0] + '.txt')  # Assuming labels are in TXT format
                if os.path.exists(label_path):
                    # Load image
                    image = cv2.imread(image_path)
                    if image is not None:
                        # Preprocess image
                        preprocessed_image = preprocess_image(image)
                        # Load label
                        with open(label_path, 'r') as f:
                            label = f.read().strip()
                        # Append preprocessed image and label to lists
                        images.append(preprocessed_image)
                        labels.append(label)
        return images, labels

    # Load images and labels for training set
    X_train, y_train = load_images_from_directory(os.path.join(train_path, 'images'), os.path.join(train_path, 'labels'))

    # Load images and labels for testing set
    X_test, y_test = load_images_from_directory(os.path.join(test_path, 'images'), os.path.join(test_path, 'labels'))

    # Load images and labels for validation set
    X_val, y_val = load_images_from_directory(os.path.join(val_path, 'images'), os.path.join(val_path, 'labels'))

    print("Number of training samples:", len(X_train))
    print("Number of testing samples:", len(X_test))
    print("Number of validation samples:", len(X_val))

    return X_train, y_train, X_test, y_test, X_val, y_val

In [7]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# SVM classifier
def train_svm(X_train, y_train):
    clf = SVC(kernel='linear')
    clf.fit(X_train, y_train)
    return clf

# KNN classifier
def train_knn(X_train, y_train):
    clf = KNeighborsClassifier(n_neighbors=5)
    clf.fit(X_train, y_train)
    return clf

# Decision tree classifier
def train_decision_tree(X_train, y_train):
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    return clf

# ANN classifier
def train_ann(X_train, y_train):
    clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
    clf.fit(X_train, y_train)
    return clf

# Function to evaluate classifier performance
def evaluate_classifier(clf, X_val, y_val):
    y_pred = clf.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

# Main function to compare classifier performance
def compare_classifiers(X_train, y_train, X_val, y_val):
    # Train SVM classifier
    svm_classifier = train_svm(X_train, y_train)
    svm_accuracy = evaluate_classifier(svm_classifier, X_val, y_val)
    print("SVM Accuracy:", svm_accuracy)

    # Train KNN classifier
    knn_classifier = train_knn(X_train, y_train)
    knn_accuracy = evaluate_classifier(knn_classifier, X_val, y_val)
    print("KNN Accuracy:", knn_accuracy)

    # Train decision tree classifier
    decision_tree_classifier = train_decision_tree(X_train, y_train)
    decision_tree_accuracy = evaluate_classifier(decision_tree_classifier, X_val, y_val)
    print("Decision Tree Accuracy:", decision_tree_accuracy)

    # Train ANN classifier
    ann_classifier = train_ann(X_train, y_train)
    ann_accuracy = evaluate_classifier(ann_classifier, X_val, y_val)
    print("ANN Accuracy:", ann_accuracy)

# Usage
# X_train, y_train, X_test, y_test, X_val, y_val = load_data(train_path, test_path, val_path)
# compare_classifiers(X_train, y_train, X_val, y_val)

In [8]:
def main():
    train_path = '/Users/mohammadadnaan/Desktop/Projects/Blood Group Detection/train'
    test_path = '/Users/mohammadadnaan/Desktop/Projects/Blood Group Detection/test'
    val_path = '/Users/mohammadadnaan/Desktop/Projects/Blood Group Detection/valid'
    
    # Load and preprocess data
    X_train, y_train, X_test, y_test, X_val, y_val = load_data(train_path, test_path, val_path)
    
    # Compare classifier performance
    compare_classifiers(X_train, y_train, X_val, y_val)

# Call main function
main()

Number of training samples: 0
Number of testing samples: 0
Number of validation samples: 0


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.